# Introducción a la vectorización

Python es un lenguaje interpretado. Esto significa que a la hora de ejecutarse un script, un intérprete lee la instrucción en tiempo real y la ejecuta. Esto ocasiona que sea un lenguaje más lento en comparación a lenguajes compilados como C.

Esta diferencia en los tiempos de ejecución es particularmente notoria cuando se ejecutan for-loops. Los lenguajes compilados pueden anticiparse a las iteraciones siguientes y guardar la memoria necesaria para ellas o en algunos casos ejecutarlas en paralelo, mientras que Python debe esperar que cada iteración termine para interpretar la siguiente. Por lo que el tiempo de ejecución resulta proporcional a n^2, siendo n la cantidad de iteraciones, mientras que en ciertos lenguajes compilados llega a ser lineal con n.

Acá aparece el término vectorización, que aplica a re-estructurar código para evitar loops explícitos en python reemplazándolos con expresiones dependientes de arrays, y/o utilizando métodos de librerías ya preparados para tal operación, como lo es Numpy, que utiliza código C ya optimizado y compilado para ejecutar determinadas operaciones.

Ejemplo simple: diferencia en tiempos de ejecución para producto vectorial entre dos vectores de dimensiones (nx1) y (1Xm), con y sin uso de loops explícitos.

![image-alt-text](https://miro.medium.com/max/720/1*BwHV54X2d6BnGi6FW1K7jw.png)

In [25]:
import numpy as np
import time

a = np.arange(10000)
b = np.arange(10000)

# usando puramente python
tic = time.process_time()
outer_product = np.zeros((10000, 10000))
for i in range(len(a)):
    for j in range(len(b)):
        outer_product[i][j]= a[i] * b[j]
toc = time.process_time()

print("python_outer_product = "+ str(outer_product))
print("Time = "+str(1000*(toc - tic ))+"ms\n")

# usando numpy
n_tic = time.process_time()
outer_product = np.outer(a, b)
n_toc = time.process_time()
print("numpy_outer_product = "+str(outer_product))
print("Time = "+str(1000*(n_toc - n_tic ))+"ms")

python_outer_product = [[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 1.0000000e+00 2.0000000e+00 ... 9.9970000e+03
  9.9980000e+03 9.9990000e+03]
 [0.0000000e+00 2.0000000e+00 4.0000000e+00 ... 1.9994000e+04
  1.9996000e+04 1.9998000e+04]
 ...
 [0.0000000e+00 9.9970000e+03 1.9994000e+04 ... 9.9940009e+07
  9.9950006e+07 9.9960003e+07]
 [0.0000000e+00 9.9980000e+03 1.9996000e+04 ... 9.9950006e+07
  9.9960004e+07 9.9970002e+07]
 [0.0000000e+00 9.9990000e+03 1.9998000e+04 ... 9.9960003e+07
  9.9970002e+07 9.9980001e+07]]
Time = 55550.4517ms

numpy_outer_product = [[       0        0        0 ...        0        0        0]
 [       0        1        2 ...     9997     9998     9999]
 [       0        2        4 ...    19994    19996    19998]
 ...
 [       0     9997    19994 ... 99940009 99950006 99960003]
 [       0     9998    19996 ... 99950006 99960004 99970002]
 [       0     9999    19998 ... 99960003 99970002 99980001]

Caso de estudio: re-sampleo de dataframe (en variable de fecha) y estimación de variables de rolling mean sobre diferentes longitud de ventana (sobre variable de target), agrupado de acuerdo a una o más variables.

In [26]:
# Librerías necesarias y conexión a azure ml workspace
from azureml.core import Datastore, Workspace, Dataset
import pandas as pd
import numpy as np
import time

ws = Workspace.from_config()
ds = Datastore(ws, name='brewdat_sandbox_131')

In [49]:
# Leyendo la data
df = pd.read_parquet("h/df.parquet")
df = df[list(df.columns)[:17]]
df = df.drop_duplicates()
df.shape

(10718210, 17)

'target_rolling_mean_n' es la media de los previos n valores de 'target' con respecto a la fecha actual, es decir, [ x(t-1) + x(t-2) + ... x(t-n)]/n . En caso de que algunos de esos n valores no exista entonces la suma de los valores previos que no son NaN se divide por dicha cantidad.

In [28]:
# Creación de Dataframe simple correspondiente a una serie temporal para dos grupos 'g1' y 'g2'
dfx = pd.DataFrame(columns=['grupo', 'fecha', 'target'])
dfx['fecha'] = ['01-01-2022', '01-03-2022', '01-04-2022', '01-05-2022', '01-06-2022', '01-07-2022', '01-08-2022', '01-09-2022', '01-02-2022', '01-03-2022', '01-04-2022', '01-05-2022', '01-06-2022', '01-09-2022']
dfx['fecha'] = pd.to_datetime(dfx['fecha'],format="%d-%m-%Y")
dfx['grupo'] = ['g1', 'g1','g1','g1','g1','g1','g1','g1','g2','g2','g2','g2','g2','g2']
dfx['target'] = np.random.randint(100, size=14)
dfx

,grupo,fecha,target
0,g1,2022-01-01,93
1,g1,2022-03-01,46
2,g1,2022-04-01,6
3,g1,2022-05-01,29
4,g1,2022-06-01,87
5,g1,2022-07-01,95
6,g1,2022-08-01,26
7,g1,2022-09-01,58
8,g2,2022-02-01,79
9,g2,2022-03-01,31


In [29]:
# Probando creación de rolling mean con ventana 2 por grupo sin resample.. resultados no correctos
dfx3 = dfx.sort_values(['grupo','fecha']).copy()
# Shift en la variable de target para utilización de pandas rolling
dfx3['target_shift'] = dfx3.groupby('grupo').target.shift()
dfx3['target_shift'] = dfx3.target_shift.where(dfx3.groupby('grupo').fecha.diff() <= '32 days', np.nan)
# Seteo de variable de fecha como índice para utilización de rolling con offset de fecha
dfx3 = dfx3.set_index('fecha')
dfx3['target_rolling_mean_2'] = dfx3.groupby('grupo').target_shift.rolling("60D", min_periods=1).mean().values
dfx3 = dfx3.reset_index()
dfx3

,fecha,grupo,target,target_shift,target_rolling_mean_2
0,2022-01-01,g1,93,NaN,NaN
1,2022-03-01,g1,46,NaN,NaN
2,2022-04-01,g1,6,46.0,46.0
3,2022-05-01,g1,29,6.0,26.0
4,2022-06-01,g1,87,29.0,17.5
5,2022-07-01,g1,95,87.0,58.0
6,2022-08-01,g1,26,95.0,91.0
7,2022-09-01,g1,58,26.0,60.5
8,2022-02-01,g2,79,NaN,NaN
9,2022-03-01,g2,31,79.0,79.0


In [ ]:
'''
t0 = time.time()
dfx['target_shiftt'] = dfx.groupby('grupo')['target'].shift(1)
t1 = time.time()
print(t1-t0)
t2 = time.time()
dfx['target_shift'] = dfx['target'].shift(1).where(dfx.grupo.eq(dfx.grupo.shift(1)))
print(time.time()-t2)
'''

In [30]:
# Probando creación de rolling mean con ventana 2 por grupo con resample..
# Resample con método de pandas
dfx2 = dfx.groupby('grupo').resample('MS', on='fecha').last().drop(columns=['grupo', 'fecha']).reset_index().copy()

dfx2.sort_values(['grupo', 'fecha'], inplace=True)
# Creando target shifteado para utilizar correctamente el rolling
dfx2['target_shift'] = dfx2['target'].shift().where(dfx2.grupo.eq(dfx2.grupo.shift())) # dfx2.groupby('grupo)['target'].shift(1)
#dfx2['target_shift'] = dfx2['target_shift].where(dfx2.grupo.eq(dfx2.grupo.shift()))
dfx2['target_rolling_mean_2b'] = dfx2.groupby('grupo')['target_shift'].rolling(window=2, min_periods=0).mean().values
dfx2

,grupo,fecha,target,target_shift,target_rolling_mean_2b
0,g1,2022-01-01,93.0,NaN,NaN
1,g1,2022-02-01,NaN,93.0,93.0
2,g1,2022-03-01,46.0,NaN,93.0
3,g1,2022-04-01,6.0,46.0,46.0
4,g1,2022-05-01,29.0,6.0,26.0
5,g1,2022-06-01,87.0,29.0,17.5
6,g1,2022-07-01,95.0,87.0,58.0
7,g1,2022-08-01,26.0,95.0,91.0
8,g1,2022-09-01,58.0,26.0,60.5
9,g2,2022-02-01,79.0,NaN,NaN


In [31]:
# Mergeo df original con el resampleado para traer las columnas calculadas solo a las fechas del df original
dfx = pd.merge(dfx[['grupo', 'fecha']], dfx2, on=['grupo', 'fecha'], how='inner')
dfx

,grupo,fecha,target,target_shift,target_rolling_mean_2b
0,g1,2022-01-01,93.0,NaN,NaN
1,g1,2022-03-01,46.0,NaN,93.0
2,g1,2022-04-01,6.0,46.0,46.0
3,g1,2022-05-01,29.0,6.0,26.0
4,g1,2022-06-01,87.0,29.0,17.5
5,g1,2022-07-01,95.0,87.0,58.0
6,g1,2022-08-01,26.0,95.0,91.0
7,g1,2022-09-01,58.0,26.0,60.5
8,g2,2022-02-01,79.0,NaN,NaN
9,g2,2022-03-01,31.0,79.0,79.0


# Trabajando con muestra de dataframe real

In [50]:
# Definiendo variables de agrupamiento y métricas sobre las que estimar las variables de rolling mean
data_level = ['customer_code', 'brand', 'size', 'mth_date']
groupby_level = ['customer_code', 'brand', 'size']
metrics = ['total_volume_hl', 'invoice_count']

df['entity_level'] = (df[groupby_level[0]].astype(str)+df[groupby_level[1]]+df[groupby_level[2]])
df_entity_level_unique_values = df['entity_level'].unique()

print("Cantidad de grupos en df completo", len(df_entity_level_unique_values))
# Defino dos dataframes con los primeros 10mil y 100mil grupos del total del df original
df_test2 = df[df['entity_level'].isin(df_entity_level_unique_values[:10000])]
df_test = df[df['entity_level'].isin(df_entity_level_unique_values[:100000])]
df_test2.to_parquet("h/df_test2.parquet")
df_test.to_parquet("h/df_test.parquet")

Cantidad de grupos en df completo 2228249


In [ ]:
'''
df_test2 = df.copy()
t0 = time.time()
df_test2 = df_test2.sort_values(data_level)
df_test2['target_shift'] = df_test2.groupby(groupby_level).total_volume_hl.shift()
df_test2['target_shift'] = df_test2.target_shift.where(df_test2.groupby(groupby_level).mth_date.diff() <= '31 days', np.nan)
df_test2 = df_test2.set_index('mth_date')
t1 = time.time()
df_test2['target_rolling_mean_2'] = df_test2.groupby(groupby_level).target_shift.rolling("60D", min_periods=0).mean().values
df_test2 = df_test2.reset_index()
t2 = time.time()
print(t2-t1, t1-t0)
'''

# Empezar desde acá cargando los dataframes de prueba!

In [ ]:
# Empezar de aquí cargando los dataframes 
import pandas as pd
import numpy as np
import time

df_test2 = pd.read_parquet("h/df_test2.parquet")
df_test = pd.read_parquet("h/df_test.parquet")

Testeando alternativa a Dataframe.resample()

In [33]:
# Utilizo como df a aquel con los primeros 10mil grupos del df original. El código de abajo es por si se quiere probar cuanto tarda la approach
# alternativa para el df original completo.
df = df_test2.copy()
'''
df = pd.read_parquet("h/df.parquet")
df = df[list(df.columns)[:17]]
df = df.drop_duplicates()
'''

'\ndf = pd.read_parquet("h/df.parquet")\ndf = df[list(df.columns)[:17]]\ndf = df.drop_duplicates()\n'

In [34]:
# Agregando columnas con la menor y mayor fecha para el grupo al que pertenece la fila correspondiente
t0 = time.time()
df['mth_date_min'] = df.groupby(groupby_level).mth_date.transform(min)
df['mth_date_max'] = df.groupby(groupby_level).mth_date.transform(max)
t1 = time.time()
print("Tiempo para mapear las fechas mín y máx por grupo a todas las filas", t1-t0)

Tiempo para mapear las fechas mín y máx por grupo a todas las filas 0.03944206237792969


In [35]:
df

,customer_commercial_region,customer_commercial_sub_region,customer_code,segment,mth_date,brand,size,product_line,sector,invoice_count,total_volume_hl,total_discount,total_net_revenue,total_gross_revenue,total_units,price,price_per_box,entity_level,mth_date_min,mth_date_max
0,Central Bajío,CMM Altiplano,100019764,42,2021-05-01,Barrilito,Media,Cerveza,Core,1,0.0780,-25.21,155.37,180.58,1.0,1991.923096,155.370000,100019764BarrilitoMedia,2021-05-01,2021-07-01
3,Central Bajío,CMM Altiplano,100037257,42,2021-05-01,Barrilito,Media,Cerveza,Core,2,0.8580,-277.31,1709.07,1986.38,11.0,1991.923096,155.370000,100037257BarrilitoMedia,2021-05-01,2022-08-01
6,Central Bajío,CMM Hidalgo,100068004,42,2021-05-01,Barrilito,Media,Cerveza,Core,1,0.3900,-126.05,776.85,902.90,5.0,1991.923096,155.370000,100068004BarrilitoMedia,2021-05-01,2022-07-01
9,Central Bajío,CMM Hidalgo,100068069,42,2021-05-01,Barrilito,Media,Cerveza,Core,1,0.3900,-126.05,776.85,902.90,5.0,1991.923096,155.370000,100068069BarrilitoMedia,2021-05-01,2021-05-01
12,Central Bajío,CMM Hidalgo,100068135,42,2021-05-01,Barrilito,Media,Cerveza,Core,1,0.3900,-175.80,801.42,977.22,6.0,1712.435913,133.570000,100068135BarrilitoMedia,2021-05-01,2021-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13440584,Central Bajío,CMM Hidalgo,101026196,43,2021-08-01,Barrilito,Media,Cerveza,Core,1,0.3900,-143.00,428.98,571.98,3.0,1833.247803,142.993333,101026196BarrilitoMedia,2021-05-01,2022-03-01
13440587,Central Bajío,CMM Metropolitana,101027184,43,2021-08-01,Barrilito,Media,Cerveza,Core,1,0.0936,0.00,190.66,190.66,1.0,2444.358887,190.660000,101027184BarrilitoMedia,2021-05-01,2022-07-01
13491243,Central Bajío,CMM Altiplano,100776062,45,2022-05-01,Barrilito,Media,Cerveza,Core,1,0.3120,-128.08,610.76,738.84,4.0,1957.564087,152.690000,100776062BarrilitoMedia,2021-05-01,2022-07-01
13571470,Central Bajío,CMM Altiplano,100776062,45,2022-07-01,Barrilito,Media,Cerveza,Core,1,0.2340,-96.06,458.07,554.13,3.0,1957.564087,152.690000,100776062BarrilitoMedia,2021-05-01,2022-07-01


Creando dataframe con todas las fechas entre las fechas mínima y máxima del dataframe total

In [36]:
# Definiendo métricas y variables de agrupamiento
data_level = ['customer_code', 'brand', 'size', 'mth_date']
metrics = ['invoice_count', 'total_volume_hl']
groupby_level = ['customer_code', 'brand', 'size']
df.sort_values(data_level, inplace=True)

# Todas las fechas desde la menor a la mayor del total del dataframe por mes
adf = pd.DataFrame(index=pd.date_range(
            df.mth_date.min(),
            df.mth_date.max(),
            freq='MS')).reset_index().rename(columns={'index':'mth_date'})
adf

,mth_date
0,2021-05-01
1,2021-06-01
2,2021-07-01
3,2021-08-01
4,2021-09-01
5,2021-10-01
6,2021-11-01
7,2021-12-01
8,2022-01-01
9,2022-02-01


Creando un dataframe con los únicos triplets correspondientes a las variables de groupby, repetidos cada uno tantas veces como la longitud del dataframe previo, y con la columna 'mth_date' correspondiente a la repetición del dataframe previo tantas veces como valores distintos de los triplets de agrupamiento.

In [37]:
# Dataframe con las variables de agrupamiento repetidas tantas veces como valores distintos entre las fechas mínima y máxima del df original total
df_entity_level = df.drop_duplicates(groupby_level)[groupby_level].reset_index(drop=True)
df_entity_level = df_entity_level.loc[df_entity_level.index.repeat(len(adf))].reset_index(drop=True)
df_entity_level['mth_date'] = np.tile(adf.mth_date.values, int(len(df_entity_level)/len(adf)))
df_entity_level

,customer_code,brand,size,mth_date
0,100019764,Barrilito,Media,2021-05-01
1,100019764,Barrilito,Media,2021-06-01
2,100019764,Barrilito,Media,2021-07-01
3,100019764,Barrilito,Media,2021-08-01
4,100019764,Barrilito,Media,2021-09-01
...,...,...,...,...
159995,101055641,Barrilito,Media,2022-04-01
159996,101055641,Barrilito,Media,2022-05-01
159997,101055641,Barrilito,Media,2022-06-01
159998,101055641,Barrilito,Media,2022-07-01


Mergeando dataframe original con dataframe previo.

In [38]:
# Mergear dataframe original con el dataframe de todas las fechas para cada grupo para traerme los valores del df original en las variables de
# métricas
df_merged = df.merge(
        df_entity_level, on=data_level, how='right')[data_level+metrics+['mth_date_min', 'mth_date_max']]
# Ordenamiento y creación de variable auxiliar para realizar el shift().where()...
df_merged.sort_values(data_level, inplace=True)
df_merged['entity_level'] = (df_merged[groupby_level[0]].astype(str)
                                    +df_merged[groupby_level[1]]+df_merged[groupby_level[2]])
df_merged

,customer_code,brand,size,mth_date,invoice_count,total_volume_hl,mth_date_min,mth_date_max,entity_level
0,100019764,Barrilito,Media,2021-05-01,1.0,0.078,2021-05-01,2021-07-01,100019764BarrilitoMedia
1,100019764,Barrilito,Media,2021-06-01,NaN,NaN,NaT,NaT,100019764BarrilitoMedia
2,100019764,Barrilito,Media,2021-07-01,1.0,0.078,2021-05-01,2021-07-01,100019764BarrilitoMedia
3,100019764,Barrilito,Media,2021-08-01,NaN,NaN,NaT,NaT,100019764BarrilitoMedia
4,100019764,Barrilito,Media,2021-09-01,NaN,NaN,NaT,NaT,100019764BarrilitoMedia
...,...,...,...,...,...,...,...,...,...
159995,101055641,Barrilito,Media,2022-04-01,NaN,NaN,NaT,NaT,101055641BarrilitoMedia
159996,101055641,Barrilito,Media,2022-05-01,NaN,NaN,NaT,NaT,101055641BarrilitoMedia
159997,101055641,Barrilito,Media,2022-06-01,NaN,NaN,NaT,NaT,101055641BarrilitoMedia
159998,101055641,Barrilito,Media,2022-07-01,NaN,NaN,NaT,NaT,101055641BarrilitoMedia


Clipeando dataframe obtenido para que cada grupo contenga todas las fechas entre las fechas mínima y máxima que originalmente tenían.

In [39]:
# Reemplazo NaT con los valores únicos por grupo de la columna correspondiente
df_merged['mth_date_min'] = df_merged.groupby(groupby_level)['mth_date_min'].transform(max)
df_merged['mth_date_max'] = df_merged.groupby(groupby_level)['mth_date_max'].transform(max)
# Variable auxiliar para seleccionar las filas que se encuentran entre la fecha mínima y máxima de cada grupo
df_merged['truth'] = np.where((df_merged.mth_date >= df_merged.mth_date_min)&(df_merged.mth_date <= df_merged.mth_date_max), True, False)
df_merged = df_merged[df_merged.truth==True]
df_merged

,customer_code,brand,size,mth_date,invoice_count,total_volume_hl,mth_date_min,mth_date_max,entity_level,truth
0,100019764,Barrilito,Media,2021-05-01,1.0,0.0780,2021-05-01,2021-07-01,100019764BarrilitoMedia,True
1,100019764,Barrilito,Media,2021-06-01,NaN,NaN,2021-05-01,2021-07-01,100019764BarrilitoMedia,True
2,100019764,Barrilito,Media,2021-07-01,1.0,0.0780,2021-05-01,2021-07-01,100019764BarrilitoMedia,True
16,100037257,Barrilito,Media,2021-05-01,2.0,0.8580,2021-05-01,2022-08-01,100037257BarrilitoMedia,True
17,100037257,Barrilito,Media,2021-06-01,NaN,NaN,2021-05-01,2022-08-01,100037257BarrilitoMedia,True
...,...,...,...,...,...,...,...,...,...,...
159989,101055641,Barrilito,Media,2021-10-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True
159990,101055641,Barrilito,Media,2021-11-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True
159991,101055641,Barrilito,Media,2021-12-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True
159992,101055641,Barrilito,Media,2022-01-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True


In [40]:
# tiempo para la approach completa de resampleo
t0 = time.time()

data_level = ['customer_code', 'brand', 'size', 'mth_date']
metrics = ['invoice_count', 'total_volume_hl']
df.sort_values(data_level, inplace=True)
groupby_level = ['customer_code', 'brand', 'size']

adf = pd.DataFrame(index=pd.date_range(
            df.mth_date.min(),
            df.mth_date.max(),
            freq='MS')).reset_index().rename(columns={'index':'mth_date'})

df_entity_level = df.drop_duplicates(groupby_level)[groupby_level].reset_index(drop=True)
df_entity_level = df_entity_level.loc[df_entity_level.index.repeat(len(adf))].reset_index(drop=True)
df_entity_level['mth_date'] = np.tile(adf.mth_date.values, int(len(df_entity_level)/len(adf)))

df_merged = df.merge(
        df_entity_level, on=data_level, how='right')[data_level+metrics+['mth_date_min', 'mth_date_max']]
df_merged.sort_values(data_level, inplace=True)
df_merged['entity_level'] = (df_merged[groupby_level[0]].astype(str)
                                    +df_merged[groupby_level[1]]+df_merged[groupby_level[2]])

df_merged['mth_date_min'] = df_merged.groupby(groupby_level)['mth_date_min'].transform(max)
df_merged['mth_date_max'] = df_merged.groupby(groupby_level)['mth_date_max'].transform(max)
df_merged['truth'] = np.where((df_merged.mth_date >= df_merged.mth_date_min)&(df_merged.mth_date <= df_merged.mth_date_max), True, False)
df_merged = df_merged[df_merged.truth==True]

t1 = time.time()
print("Tiempo total de resampleo alternativo para 10000 grupos", t1-t0)

Tiempo total de resampleo alternativo para 10000 grupos 0.45566463470458984


Testeando con resampleo alternativo, rolling + mean funciones de pandas

In [41]:
import warnings
warnings.filterwarnings("ignore")

df_merged2 = df_merged.copy()
# Target shifteado sobre el que calcular el rolling mean
df_merged.loc[:,'target_shift'] = df_merged['invoice_count'].shift(1).where(df_merged.entity_level.eq(df_merged.entity_level.shift(1)))
t0 = time.time()
# Rolling mean por grupo con métodos de pandas
df_merged.loc[:,'target_mean_2'] = df_merged.groupby('entity_level')['target_shift'].rolling(window=2, min_periods=0).mean().values
t1 = time.time()

In [42]:
print("Tiempo para ejecución de rolling mean en df resampleado alternativamente", t1 - t0)

Tiempo para ejecución de rolling mean en df resampleado alternativamente 0.8401856422424316


Resampleo usando resample() pandas

In [43]:
tx0 = time.time()
df_test2.sort_values(data_level, inplace=True)
# Resampleo con método de pandas en la variable de fecha, por grupo.
dff = df_test2.groupby('entity_level').resample('MS', on='mth_date').last().drop(columns=['mth_date', 'entity_level']).reset_index()
print("Tiempo necesario para resamplear dataframe con pandas.Dataframe.resample()", time.time()-tx0)

Tiempo necesario para resamplear dataframe con pandas.Dataframe.resample() 46.49331092834473


In [44]:
# Ordenando df en variables de grupo y fecha para aplicar el shift().where()
dff.sort_values(['entity_level', 'mth_date'], inplace=True)
dff = dff[['entity_level', 'mth_date']+metrics]
# Target shifteado sobre el que calcular el rolling mean
dff.loc[:,'target_shift'] = dff['invoice_count'].shift(1).where(dff.entity_level.eq(dff.entity_level.shift(1)))
tx0 = time.time()
# Rolling mean por grupo con métodos de pandas
dff.loc[:,'target_mean_2'] = dff.groupby('entity_level')['target_shift'].rolling(window=2, min_periods=0).mean().values
print("Tiempo para ejecución de rolling mean en df resampleado con método de pandas", time.time()-tx0)

Tiempo para ejecución de rolling mean en df resampleado con método de pandas 0.8186922073364258


In [45]:
# Printing total number of files
print("Dimensiones de los DFs resultantes de resampleo con resample() y resampleo alternativo")
print(df_merged.shape, dff.shape)

Dimensiones de los DFs resultantes de resampleo con resample() y resampleo alternativo
(101424, 12) (101424, 6)


Chequeo si se obtienen los mismos resultados

In [46]:
print("DFs resultantes de resampleo con resample() y resampleo alternativo, son iguales?", df_merged[list(dff.columns)].sort_values(
    ['entity_level', 'mth_date']).reset_index(drop=True).equals(dff.sort_values(['entity_level', 'mth_date']).reset_index(drop=True)))

DFs resultantes de resampleo con resample() y resampleo alternativo, son iguales? True


# Alternativa a rolling mean

In [47]:
df_merged2 = df_merged.copy()
window_list = [2]
metric = 'invoice_count'
w = window_list[0]

t0 = time.time()
for i in range(1,max(window_list)+1):
    df_merged2[f'{metric}_shifted_{str(i)}'] = df_merged2[
        metric].shift(i).where(df_merged2.entity_level.eq(df_merged2.entity_level.shift(i)), np.nan, ) # df.groupby()['target'].shift()

df_merged2[f'{metric}_mean_{str(w)}'] = (df_merged2[[f'{metric}_shifted_{str(i)}'
                                                            for i in range(1,w+1)]].mean(axis=1))
print("Tiempo necesario para estimar rolling mean con vectorización", time.time()-t0)
df_merged2

Tiempo necesario para estimar rolling mean con vectorización 0.017359495162963867


,customer_code,brand,size,mth_date,invoice_count,total_volume_hl,mth_date_min,mth_date_max,entity_level,truth,target_shift,target_mean_2,invoice_count_shifted_1,invoice_count_shifted_2,invoice_count_mean_2
0,100019764,Barrilito,Media,2021-05-01,1.0,0.0780,2021-05-01,2021-07-01,100019764BarrilitoMedia,True,NaN,NaN,NaN,NaN,NaN
1,100019764,Barrilito,Media,2021-06-01,NaN,NaN,2021-05-01,2021-07-01,100019764BarrilitoMedia,True,1.0,1.0,1.0,NaN,1.0
2,100019764,Barrilito,Media,2021-07-01,1.0,0.0780,2021-05-01,2021-07-01,100019764BarrilitoMedia,True,NaN,1.0,NaN,1.0,1.0
16,100037257,Barrilito,Media,2021-05-01,2.0,0.8580,2021-05-01,2022-08-01,100037257BarrilitoMedia,True,NaN,NaN,NaN,NaN,NaN
17,100037257,Barrilito,Media,2021-06-01,NaN,NaN,2021-05-01,2022-08-01,100037257BarrilitoMedia,True,2.0,2.0,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159989,101055641,Barrilito,Media,2021-10-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True,NaN,NaN,NaN,NaN,NaN
159990,101055641,Barrilito,Media,2021-11-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True,NaN,NaN,NaN,NaN,NaN
159991,101055641,Barrilito,Media,2021-12-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True,NaN,NaN,NaN,NaN,NaN
159992,101055641,Barrilito,Media,2022-01-01,NaN,NaN,2021-05-01,2022-02-01,101055641BarrilitoMedia,True,NaN,NaN,NaN,NaN,NaN


In [48]:
columns = data_level + [f'{metric}_mean_{str(w)}']
t0 = time.time()
df = df.merge(df_merged2[columns], on=data_level, how='left')
print("Tiempo necesario para mergear df original con el de sampleo alternativo", time.time()-t0)
#df

Tiempo necesario para mergear df original con el de sampleo alternativo 0.0645759105682373


Ejercicio 1: Dado un dataframe con x cantidad de grupos (definidos por 'customer_code', 'brand', 'size'), se desea eliminar todos aquellos grupos cuyo máximo valor de la variable 'total_volume_hl' es menor a THRESHOLD. 
Hint: Dataframe.transform(max) está optimizado para trabajar por grupo.

In [ ]:
THRESHOLD = 0.0790
df_ejercicio = df[['customer_code', 'brand', 'size', 'mth_date', 'total_volume_hl']]
df_ejercicio

Ejercicio 2: Siguiendo la misma idea de agrupamiento, calcular el valor medio y la suma de la variable de target, por grupo, para aquellos valores dentro de cada grupo que se encuentren entre la fecha máxima por grupo y una mínima dada como la máxima mencionada - 6 meses. Hint: chequear métoodo agg().